<a href="https://colab.research.google.com/github/Rahul-Gadadhar/lab/blob/main/lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 3rd experiment much shorter version with output from chatgpt

import socket
import threading

SERVER PROGRAM:

def handle_client(conn):
    recv_buffer = ''
    missing = []
    expected = 1
    buffer_size = 8192

    try:
        while True:
            data = conn.recv(buffer_size).decode().split()
            if not data:
                break
            for seq in map(int, data):
                if seq == expected:
                    expected += 1
                elif seq in missing:
                    missing.remove(seq)
                else:
                    missing.append(expected)
                    expected += 1
                recv_buffer += str(seq)
                conn.sendall((str(expected) + ' ').encode())

    except:
        pass
    finally:
        conn.close()


def server():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind((socket.gethostname(), 4001))
    sock.listen(5)
    print("Server Running ......")

    while True:
        conn, addr = sock.accept()
        print('Connected to:', addr)
        threading.Thread(target=handle_client, args=(conn,)).start()

if __name__ == '__main__':
    server()


CLIENT PROGRAM:

import socket
import random
import time
from collections import deque



# Constants
TOTAL_PACKETS = 100
PACKET_SIZE = 4
DROP_PROBABILITY = 0.01


def connect():
    host = socket.gethostname()
    port = 4001
    try:
        conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn.connect((host, port))
        return conn

    except Exception as e:
        print("Error:", str(e))
        return None


def process_packets(conn):
    seq_num = 0
    pkt_success_sent = 0
    sent_count = 0
    window_size = 1
    dropped_pkt = deque()

    while pkt_success_sent < TOTAL_PACKETS:
        to_send = []

        for _ in range(window_size):
            seq_num += 1
            to_send.append(seq_num)

        for pkt in to_send:
            sent_count += 1
            if random.random() < DROP_PROBABILITY:
                dropped_pkt.append(pkt)
            else:
                conn.sendall(f"{pkt} ".encode())
                pkt_success_sent += 1

        try:
            ack_str = conn.recv(8192).decode()
            acks = list(map(int, ack_str.split()))
            for ack in acks:
                if ack and window_size < 2048:
                    window_size += 1
        except socket.timeout:
            pass

        if dropped_pkt:
            window_size = max(1, window_size // 2)

        if sent_count % 10 == 0:
            print(f"{sent_count} packets sent...")

    print("Execution Completed")
    print(f"Packets sent: {sent_count}")
    conn.close()


def main():

    start_time = time.time()
    conn = None


    while conn is None:
        conn = connect()
        time.sleep(1)

    if conn:
        process_packets(conn)

    end_time = time.time()
    print("Runtime:", end_time - start_time)


if __name__ == "__main__":
    main()